# Portfolio Analysis with Statistics

To access stock data, we will use the [yfinance library](https://github.com/ranaroussi/yfinance). We first download the historical monthly stock prices for the chosen stocks/tickers(slightly modifying the code in the library tutorial). The data comes in the form of a pandas dataframe with multi-level headers, so we also unstack the levels for simpler access.

In [19]:
import yfinance as yf
import numpy as np
import scipy.stats as stats
import pandas as pd
ticks = ["AMD", "GE", "MSFT", "PILL", "TQQQ", "XLV", "TSLA"]
#ticks = pd.read_csv("nasdaq_screener.csv")['Symbol'].tolist()
data = yf.download(tickers = ['^GSPC'] + ticks, start = '2016-06-01', interval = "1mo", group_by = 'ticker', auto_adjust = True, threads = True)

[*********************100%***********************]  8 of 8 completed


In [20]:
# unstack
stocks_raw = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
stocks_raw = stocks_raw.groupby('Ticker').filter(lambda x: len(x) > 36)
stocks_raw = stocks_raw.sort_values(by=['Date', 'Ticker'])
stocks_raw

,Ticker,Close,High,Low,Open,Volume
Date,,,,,,
2016-06-01,AMD,5.140000,5.520000,4.070000,4.600000,6.256425e+08
2016-06-01,GE,27.539967,27.557463,25.519085,26.280196,9.001391e+08
2016-06-01,MSFT,47.140781,48.780623,44.257246,48.310780,8.236270e+08
2016-06-01,TQQQ,8.010098,8.784329,6.802596,8.611354,7.897104e+08
2016-06-01,TSLA,42.456001,48.169998,37.574001,44.296001,6.083640e+08
...,...,...,...,...,...,...
2021-06-28,PILL,24.980000,25.230000,24.775000,25.230000,8.524000e+03
2021-06-28,TQQQ,121.669998,121.879997,118.750000,118.769997,2.219692e+07
2021-06-28,TSLA,688.719971,694.699890,670.320007,671.640015,2.162816e+07


Since we only want one stock price per month, we filter out the last row of each asset if it does not fall on the first day of the month. We also only keep the last 5 years of data to maintain an accurate representation of each company's relevant returns(profitability in the 1990s does not entail profitability in 2020s). Because we require the previous month's price to compute the current month's return, we need to keep an extra month(a total of 61 months)

In [3]:
recent = stocks_raw.index[-1] - pd.DateOffset(day = 1)
stocks = stocks_raw.loc[stocks_raw.index <= recent].copy()
#sp500 = sp500_raw.loc[(sp500_raw.index <= recent) & (sp500_raw.index >= begin)].copy()

To compute the (percent) return of a specified observation, we subtract the current price with last month's price and divide by last month's price. We can easily vectorize this by subtracting an array of the (open) prices without the last observation from an array of the (open) prices without the first observation. We then divide by the former.  

In [4]:
pivoted = stocks.pivot(columns = 'Ticker', values = 'Open').dropna(axis=1, how='all')
spprices = pivoted['^GSPC'].to_numpy()
pivoted = pivoted.drop(columns = ['^GSPC'])
prices = pivoted.to_numpy()
returnarr = prices[1:, :] - prices[:(prices.shape[0] - 1), :]
returnarr = returnarr /  prices[:(prices.shape[0] - 1), :]
spreturns = (spprices[1:] - spprices[:(len(spprices) - 1)]) / spprices[:(len(spprices) - 1)]

In [5]:
returnarr

array([[ 1.06521795e-01,  5.58750705e-02, -2.49811888e-02,
                    nan, -7.14630408e-02, -6.92613485e-02,
         1.44713765e-02],
       [ 3.53634511e-01, -1.14357393e-02,  1.06982353e-01,
                    nan,  2.30369124e-01,  1.42427422e-01,
         4.73075212e-02],
       [ 4.20899808e-02,  9.63825444e-04,  1.35217101e-02,
                    nan,  2.80643080e-02, -1.12484087e-01,
        -3.33466744e-02],
       [-3.20334296e-02, -4.68724210e-02,  7.01631633e-03,
                    nan,  5.36918504e-02,  1.57408970e-02,
        -9.51141703e-03],
       [ 5.32374637e-02, -1.56142953e-02,  4.45915167e-02,
                    nan, -3.66757012e-02, -6.71690400e-02,
        -6.11507081e-02],
       [ 2.18579217e-01,  5.65518449e-02,  9.10571644e-03,
                    nan, -5.67167708e-04, -4.94344602e-02,
         2.12168793e-02],
       [ 2.80269056e-01,  4.13950063e-02,  4.45850110e-02,
                    nan,  5.04943953e-02,  1.41354539e-01,
         1.1391225

To make calculating parameters easier, we can pivot the dataframe such that each ticker's percent returns form individual columns. Note that we need to mask the data matrix to ignore NaN values. 

In [6]:
#stockpivot = stocks.pivot(columns = 'Ticker', values = 'PercReturns').iloc[1:].dropna(axis=1, how='all')
#returnarr = stockpivot.to_numpy()
#returnarr = returnarr[:, ~np.isnan(returnarr).all(axis=0)]
#print(returnarr)
# mask to ignore NaN for relatively new stocks
#returnmean = np.ma.masked_invalid(returnarr).mean(axis = 0).data
returnmean = returnarr.mean(axis = 0)
#returncov = np.ma.cov(np.ma.masked_invalid(returnarr), rowvar = False).data
#invcov = np.linalg.lstsq(a = returncov, b = np.eye(len(ticks)), rcond = None)[0]

In [7]:
betas = np.zeros(returnarr.shape[1])
alphas = np.zeros(returnarr.shape[1])
unsyserr = np.zeros(returnarr.shape[1])
for i in np.arange(returnarr.shape[1]):
    treturn = returnarr[:,i]
    tnonan = treturn[np.logical_not(np.isnan(treturn))]
    spmatch = spreturns[(len(spreturns) - len(tnonan)):]
    try:
        betas[i], alphas[i], r, p, se = stats.linregress(spmatch, tnonan)
    except: 
        print(returnarr[:,i])
    unsyserr[i] = np.sum((tnonan - alphas[i] - betas[i]*spmatch)**2) / (len(spmatch) - 2)

In [8]:
pivoted.columns.values

array(['AMD', 'GE', 'MSFT', 'PILL', 'TQQQ', 'TSLA', 'XLV'], dtype=object)

In [9]:
simdf = pd.DataFrame(data = {'alpha': alphas, 'beta': betas, 'eps': unsyserr, 'rmean': returnmean}, index = pivoted.columns.values)
simdf['excess'] = simdf['rmean'] / simdf['beta']
simdf = simdf.sort_values(by=['excess'], ascending = False)
simdf = simdf.loc[(simdf['excess'] > 0) & (simdf['beta'] > 0)]

In [10]:
simdf

,alpha,beta,eps,rmean,excess
MSFT,0.018721,0.806221,0.001104,0.029107,0.036104
AMD,0.036896,1.936005,0.019382,0.061837,0.031941
TSLA,0.034843,2.099968,0.029760,0.061896,0.029475
TQQQ,0.014494,3.314010,0.003893,0.057187,0.017256
XLV,0.001731,0.771602,0.000578,0.011671,0.015125


In [11]:
num = simdf['rmean'] * simdf['beta'] / simdf['eps']
den = simdf['beta']**2 / simdf['eps']
simdf['C'] = spreturns.var() * num.cumsum() / (1 + spreturns.var() * den.cumsum())

In [12]:
cutoff = simdf.loc[simdf['C'] < simdf['excess']]
z = (cutoff['beta'] / cutoff['eps']) * (cutoff['excess'] - cutoff['C'])

In [13]:
#pd.set_option('display.max_rows', 110)
z.sort_values(ascending = False) / z.sum()

MSFT    0.889953
AMD     0.074992
TSLA    0.035054
dtype: float64

In [14]:
len(z)

3